In [1]:
import numpy as np
import random

In [2]:
class Lattice:
    def __init__(self,L,beta,start='cold',J=-1):
        self.J = J
        self.L = L
        self.N = L*L
        self.beta = beta
        self.metropolisProb = [np.exp(-2*beta*x) for x in range(2,5,2)]
        self.wolffProb = 1 - np.exp(-2*beta*J)
        if start == 'hot':
            self.sites = np.random.choice((-1,1),(L,L))
        elif start == 'cold':
            self.sites = np.ones((L,L),dtype='int8')
        else:
            print("'{}' is an incorrect start configuration.  Valid choices are 'hot' or 'cold'.".format(start))
            print("Default to 'cold' start")
            self.sites = np.ones((L,L),dtype='int8')
            
    def update_metropolis(self):
        for i in range(self.N):
            x = random.randrange(0,self.L)
            y = random.randrange(0,self.L)
            sumNeighbor  = self.sites[(y+1)%self.L][x]
            sumNeighbor += self.sites[(y-1)%self.L][x]
            sumNeighbor += self.sites[y][(x+1)%self.L]
            sumNeighbor += self.sites[y][(x-1)%self.L]
                
            delta = self.sites[y][x]*sumNeighbor
                                
            if delta <= 0:
                self.sites[y][x] *= -1
            elif random.random()<self.metropolisProb[int(delta/2-1)]:
                self.sites[y][x] *= -1
                            
    def update_wolff(self):
        cluster = np.full((self.L,self.L),False,dtype='bool')
        
        X = random.randrange(0,self.L)
        Y = random.randrange(0,self.L)
        
        def tryAdd(x,y,clusterSpin,cluster):
            if self.sites[y][x] == clusterSpin:
                if random.random() < self.wolffProb:
                    growCluster(x,y,clusterSpin,cluster)
        
        def growCluster(x,y,clusterSpin,cluster):
            self.sites[y][x] *= -1
            cluster[y][x] = True
            
            if cluster[(y+1)%self.L][x] == False:
                tryAdd(x,(y+1)%self.L,clusterSpin,cluster)
            if cluster[(y-1)%self.L][x] == False:
                tryAdd(x,(y-1)%self.L,clusterSpin,cluster)
            if cluster[y][(x+1)%self.L] == False:
                tryAdd((x+1)%self.L,y,clusterSpin,cluster)
            if cluster[y][(x-1%self.L)] == False:
                tryAdd((x-1)%self.L,y,clusterSpin,cluster)
                
        growCluster(X,Y,self.sites[Y][X],cluster)

    def m(self):
        return self.sites.sum()/self.N
    
    def e(self):
        e = 0
        for x in range(self.L):
            for y in range(self.L):
                sumNeighbor  = self.sites[(y+1)%self.L][x]
                sumNeighbor += self.sites[(y-1)%self.L][x]
                sumNeighbor += self.sites[y][(x+1)%self.L]
                sumNeighbor += self.sites[y][(x-1)%self.L]
                
                e += self.sites[y][x]*sumNeighbor
        return e/self.N
    
    def chi(self):
        pass
    
    def c(self):
        pass
    
    def __str__(self):
        string = "Mean Magnetization Per Site:\t{}\n".format(self.m())
        string += "Mean Energy Per Site:\t\t{}\n".format(self.e())
        string += str(self.sites)
        return string



In [11]:
sim = Lattice(10,100,'hot')
print(sim)
for i in range(1000):
    sim.update_wolff()
print(sim)


Mean Magnetization Per Site:	-0.12
Mean Energy Per Site:		-0.08
[[-1  1  1 -1 -1 -1  1 -1 -1  1]
 [ 1  1  1 -1  1 -1  1 -1  1  1]
 [ 1 -1  1  1 -1 -1 -1  1  1 -1]
 [ 1  1 -1 -1 -1 -1 -1 -1 -1  1]
 [-1  1 -1 -1 -1  1  1 -1 -1 -1]
 [-1  1 -1 -1 -1  1  1 -1 -1  1]
 [-1 -1  1  1  1 -1 -1  1 -1  1]
 [-1 -1  1 -1  1 -1 -1  1 -1  1]
 [-1  1 -1  1  1  1  1 -1  1 -1]
 [ 1 -1 -1 -1  1  1 -1 -1 -1 -1]]
Mean Magnetization Per Site:	0.12
Mean Energy Per Site:		0.16
[[ 1  1 -1  1  1  1 -1  1  1  1]
 [ 1  1 -1 -1 -1  1 -1  1  1  1]
 [ 1 -1 -1  1 -1 -1  1 -1 -1  1]
 [-1  1  1  1 -1  1 -1 -1 -1  1]
 [-1 -1 -1 -1  1 -1 -1 -1 -1 -1]
 [ 1  1  1 -1  1 -1  1  1 -1  1]
 [-1  1  1 -1  1  1  1  1 -1  1]
 [ 1  1 -1  1  1  1  1  1 -1  1]
 [ 1 -1 -1  1  1 -1  1  1 -1  1]
 [-1  1  1 -1 -1 -1  1 -1 -1  1]]
